In [2]:
import json
import random
import malaya

2023-07-26 14:50:41.597955: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 14:50:41.764918: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-26 14:50:42.679885: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-26 14:50:42.679939: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [3]:
language_detection = malaya.language_detection.fasttext(model = 'mesolitica/fasttext-language-detection-ms-id')

In [4]:
template_malaya = {
    'description': 'Template used by Malaya.',
    'prompt_input': 'Di bawah ialah arahan yang menerangkan tugasan, termasuk dengan input yang menyediakan konteks lanjut. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Input:\n{input}\n\n### Jawapan:\n',
    'prompt_no_input': 'Di bawah ialah arahan yang menerangkan tugasan. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Jawapan:\n',
    'response_split': '### Jawapan:',
}

In [7]:
language_detection.predict([a])

['standard-malay']

In [8]:
from tqdm import tqdm

data = []
with open('/home/husein/ssd3/gpt4all-1.3/translated-gpt4all-filtered-noncode.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        try:
            a = l['r'][-1]['result']
            
            if language_detection.predict([a])[0] != 'standard-malay':
                continue
            
            ori_q = l['src']['prompt']
            
            if random.random() > 0.7:
                q = ori_q
            else:
                q = l['r'][0]['result']
                if language_detection.predict([q])[0] != 'standard-malay':
                    continue
            
            o = {
                'prompt_input': template_malaya["prompt_no_input"],
                'input': q,
                'output': a
            }
            data.append(o)
            
        except Exception as e:
            pass

439509it [00:50, 8663.00it/s] 


In [9]:
len(data)

416856

In [11]:
with open('prepared-gpt4all-v1.3.jsonl', 'w') as fopen:
    for l in data:
        fopen.write(f'{json.dumps(l)}\n')